In [ ]:
py_path = !which python
py_version = !python --version
print("Python: version {} under {}".format(py_version, py_path))

import torch
print("PyTorch: version {}".format([torch.__version__]))

In [ ]:
from torch.utils.cpp_extension import CUDA_HOME
from torch.utils.cpp_extension import CppExtension
from torch.utils.cpp_extension import CUDAExtension

#print(CUDA_HOME)
#print(CppExtension)
#print( dir(CUDAExtension) )

#import setuptools
#dir(setuptools)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np

import requests
from io import BytesIO
from PIL import Image
from time import perf_counter

In [ ]:
def load(url):
    response = requests.get(url)
    pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img):
    pylab.rcParams['figure.figsize'] = 20, 12
    plt.imshow(img[:, :, [2, 1, 0]])
    #plt.axis("off")

In [ ]:
from maskrcnn_benchmark.config import cfg

config_file = "/home/fenli/my_code/maskrcnn-benchmark/configs/myexp.yaml"
cfg.merge_from_file(config_file)

GpuIdToUse = None
#GpuIdToUse = "0,1"

if GpuIdToUse is not None:
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = GpuIdToUse
    print ("using gpu")
    cfg.merge_from_list(["MODEL.DEVICE", "cuda"])
else:
    print ("using cpu")
    cfg.merge_from_list(["MODEL.DEVICE", "cpu"])

    
cfg.merge_from_list(["MODEL.WEIGHT", "/home/fenli/my_model/fashion_category_attr/model_FasterRCnn_R50FPN_iter80000.pth"])

In [ ]:
from mypredictor import MyDemo

start_time = perf_counter()
my_demo = MyDemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.1,
)
elapsed_time = perf_counter() - start_time
print("PERF: Model loaded in {:.0f} ms".format(elapsed_time*1000))

print(cfg)

In [ ]:
#model = my_demo.get_model()
#torch.save(model, "/home/fenli/my_model/fashion_category_attr/saved_model.pth")

model = torch.load("/home/fenli/my_model/fashion_category_attr/saved_model.pth")
my_demo.set_model(model)
print(model)

In [ ]:
image = load(
#    "https://cdn.cliqueinc.com/cache/posts/245509/japanese-street-style-245509-1513874225144-image.700x0c.jpg"
#    "https://static01.nyt.com/images/2019/02/27/fashion/weddings/27doozy1/27doozy1-superJumbo-v2.jpg"
#    "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/hbz-street-style-ss2019-nyfw-day4-12-1536604387.jpg"
#    "https://assets.vogue.com/photos/591daa7caf3eb104a5ac3ac5/master/pass/15-sydney-resort-18-day-5.jpg"
#    "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/milan-mfw-street-style-ss18-day-4-tyler-joe-023-1506598430.jpg"
#     "https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/hbz-street-style-copenhagen-day3-06-1502740501.jpg" # 3
#    "https://media.gq.com/photos/5894caecfa95131655778571/master/pass/0202-dan-roberts-street-style-NYMFW-FW17-30.jpg"
    "https://timedotcom.files.wordpress.com/2018/09/neon-brights-street-style.jpg"
#    "https://s3-eu-west-2.amazonaws.com/pausemagbucket/wp-content/uploads/2018/03/PARIS-A_W-18_19-D5_0122-copy-1024x683.jpg"
#    "https://uproxx.files.wordpress.com/2018/10/tokyo-fashion-week-street-style-feat.jpg"
)
imshow(image)

In [ ]:
start_time = perf_counter()
predictions = my_demo.run_on_opencv_image(image, 0.2)
elapsed_time = perf_counter() - start_time
print("PERF: Model executed in {:.0f} ms".format(elapsed_time*1000))
imshow(predictions)
